

* Loading preprocessed data
* Train Logistic regression
* Train Decision Tree classifier
* Traing XGBoost classifier
* Train LightGBM classifier
* Train Random Forest classifier


Declare feature vector and target variable

Split data into separate training and test set

Train the XGBoost classifier

Make predictions with XGBoost classifier

Check accuracy score

k-fold Cross Validation using XGBoost

Feature importance with XGBoost

Results and conclusion

### Imports

In [2]:
import opendatasets as od
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

### Loading preprocessed data

In [6]:
target_col = 'RainTomorrow'

train_inputs = pd.read_parquet('data/parquet/train_inputs.parquet')
val_inputs = pd.read_parquet('data/parquet/val_inputs.parquet')
test_inputs = pd.read_parquet('data/parquet/test_inputs.parquet')

train_targets = pd.read_parquet('data/parquet/train_targets.parquet')[target_col]
val_targets = pd.read_parquet('data/parquet/val_targets.parquet')[target_col]
test_targets = pd.read_parquet('data/parquet/test_targets.parquet')[target_col]

In [7]:
print('train_inputs:', train_inputs.shape)
print('train_targets:', train_targets.shape)
print('val_inputs:', val_inputs.shape)
print('val_targets:', val_targets.shape)
print('test_inputs:', test_inputs.shape)
print('test_targets:', test_targets.shape)

train_inputs: (97988, 123)
train_targets: (97988,)
val_inputs: (17089, 123)
val_targets: (17089,)
test_inputs: (25710, 123)
test_targets: (25710,)


### `LogisticRegression`

### `Decision Tree classifier`


### `XGBoost`

### `LightGBM`

### `Random Forest`